In [1]:
import gradio as gr
from dotenv import load_dotenv
import tomli
from link_converter import get_file_url
from pathlib import Path

load_dotenv()


def load_config():
    with open("config.toml", "rb") as f:
        config = tomli.load(f)
    return config


CONFIG = load_config()


In [2]:
BASE_DIR = CONFIG["directories"]["base_dir"]
BASE_URL = CONFIG["directories"]["base_url"]
model = CONFIG["model"]["gemini_model_name"]

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

system_message = "Answer the questions about Montaigne."


class Chatbot:
    def __init__(self, model: str, system_message: str):
        self.llm = ChatGoogleGenerativeAI(model=model)
        self.system_message = [{"role": "system", "content": system_message}]

    def _history_to_messages(self, history: list):
        # because history comes from gradio.chatinterface, it has more keys than just role and content
        # Therefore we need to filter out the extra keys to be compatible with langchain
        return [{k: v for k, v in msg.items() if k in ["role", "content"]} for msg in history]

    def query_llm(self, query: str, history: list):
        history_langchain = self.system_message + self._history_to_messages(history)
        history_langchain.append({"role": "user", "content": query})
        response = self.llm.invoke(history_langchain)

        return response.content


In [16]:
current_file_path = None
current_url = None


def get_url(file_path: str):
    global current_file_path
    global current_url
    current_file_path = Path(file_path)
    current_url = get_file_url(current_file_path, BASE_URL)
    return current_file_path.name, current_url

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            fe = gr.FileExplorer(glob="**/*.pdf", root_dir=BASE_DIR, file_count="single")
        with gr.Column():
            url = gr.Textbox(label="URL")
            path = gr.Textbox(label="File Path")

            chat_interface = gr.ChatInterface(
                fn=query_llm,
                type="messages",
                title="Montaigne Chatbot",
                description="Ask questions about Montaigne's works.",
            )

    fe.change(fn=get_url, inputs=fe, outputs=[path, url])


demo.launch()


In [17]:
chatbot = Chatbot(model=model, system_message=system_message)

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            fe = gr.FileExplorer(glob="**/*.pdf", root_dir=BASE_DIR, file_count="single")
        with gr.Column():
            url = gr.Textbox(label="URL")
            path = gr.Textbox(label="File Path")

            chat_interface = gr.ChatInterface(
                fn=chatbot.query_llm,
                type="messages",
                title="Montaigne Chatbot",
                description="Ask questions about Montaigne's works.",
            )

    fe.change(fn=get_url, inputs=fe, outputs=[path, url])


demo.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
